In [39]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from pathlib import Path

def all_files(dirpath):
    return [str(path) for path in Path(dirpath).glob("*.txt")]

def imdb_dataset(filepaths_positive, filepaths_negative, n_read_threads=5):
    dataset_neg = tf.data.TextLineDataset(filepaths_negative, num_parallel_reads=n_read_threads)
    dataset_neg = dataset_neg.map(lambda review: (review, 0))
    
    dataset_pos = tf.data.TextLineDataset(filepaths_positive, num_parallel_reads=n_read_threads)
    dataset_pos = dataset_pos.map(lambda review: (review, 1))
    
    return tf.data.Dataset.concatenate(dataset_pos, dataset_neg)

train_pos = all_files("./datasets/aclImdb/train/pos/")
train_neg = all_files("./datasets/aclImdb/train/neg/")
test_pos = all_files("./datasets/aclImdb/test/pos/")
test_neg = all_files("./datasets/aclImdb/test/neg/")

full_train_data = imdb_dataset(train_pos, train_neg).cache().shuffle(25000)
val_data = full_train_data.take(5000)
train_data = full_train_data.skip(5000)
test_data = imdb_dataset(test_pos, test_neg).cache()

In [40]:
val_data = val_data.batch(32).prefetch(1)
train_data = train_data.batch(32).prefetch(1)
test_data = test_data.batch(32).prefetch(1)

In [46]:
simple_example = tf.constant([[1, 3, 1, 0, 0], [2, 2, 0, 0, 0]])
tf.reduce_sum(tf.one_hot(simple_example, 4), axis=1)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2., 2., 0., 1.],
       [3., 0., 2., 0.]], dtype=float32)>